In [4]:
pip install pytorch_lightning

Python(73761) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.



[notice] A new release of pip is available: 24.3.1 -> 25.2
[notice] To update, run: python3.10 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [5]:
import numpy as np
from glob import glob
from tqdm import tqdm 
from model import VAE
import torch
import cv2
import json 


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device, torch.cuda.is_available())


cpu False


In [7]:

latentDim = 25
checkpoint_path = "./ckpt_files/anar_lat_25_052324.ckpt"
checkpoint = torch.load(checkpoint_path, map_location=torch.device('cpu'))
model = VAE(latentDim)
model.load_state_dict(checkpoint['state_dict'])
model = model.to(device)
model.eval()


VAE(
  (encoder): Encoder(
    (conv_stack): Sequential(
      (0): Conv2d(1, 32, kernel_size=(11, 11), stride=(1, 1), padding=same)
      (1): ReLU()
      (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (3): Conv2d(32, 64, kernel_size=(5, 5), stride=(1, 1), padding=same)
      (4): ReLU()
      (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (6): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=same)
      (7): ReLU()
      (8): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (9): Flatten()
      (10): Linear(in_features=8192, out_features=50, bias=True)
    )
  )
  (decoder): Decoder(
    (inverse_conv_stack): Sequential(
      (0): Linear(in_features=25, out_features=1024, bias=True)
      (1): ReLU()
      (2): UnFlatten()
      (3): ConvTranspose2d(16, 128, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
      (4): ReLU()
      (5): ConvTranspose2d(128, 64, ke

In [8]:
# Read data from string 
def process_string_mech(dir, toNpy = True):
    # I do not know why but this works for windows os. You may need to change this if you are using linux/macbook
    # Zhijie: you can test using strings like: 
    # ./outputs-4bar/-0.001 2.728 5.504 -1.565 -5.632 -2.481 -8.711 9.682 1.320 -5.630 -7.171 3.601 RRRP  0.42 0.026 0.732 -0.026 0.42 2.011 0. 0. 1. .jpg
    input_string = dir.split('\\')[-1].split('.j')[0] 
    
    # Split the string by spaces
    parts = input_string.split()
    
    # Initialize lists to hold floats
    floats_before = []
    floats_after = []
    letter_string = None
    
    # Iterate over parts to separate floats and the letter string
    for part in parts:
        try:
            # Try to convert part to float
            num = float(part)
            # Add to floats_before if letter_string is not yet found
            if letter_string is None:
                floats_before.append(num)
            else:
                floats_after.append(num)
        except ValueError:
            # If conversion fails, this part is the letter string
            letter_string = part
    
    if toNpy:
        floats_before = np.array(floats_before).reshape((-1, 2))
        floats_after = np.matrix(floats_after).reshape((3, 3))
    
    #if len(floats_before) != 10: # security check... you should change this for your specific mechanism. 
    #    print('you got fucked', dir, '\n' , floats_before, '\n')
    return floats_before, letter_string, floats_after

In [14]:
# BSIdictionary update (PRPR)
file_path = './KV_468_062324.json'

# Open and read the JSON file
with open(file_path, 'r') as file:
    KVdict = json.load(file) 

image_folder = './outputs-6bar/'

setSize = 3000 # len(imgStrings) # determine how many samples for each type. 
batchSize = 1000
four_bar = ['RRRR', 'RRRP', 'RRPR', 'PRPR']  # 
six_bar  = ['Watt1T1A1', 'Watt1T2A1', 'Watt1T3A1', 'Watt1T1A2', 'Watt1T2A2', 'Watt1T3A2', 
            'Watt2T1A1', 'Watt2T2A1', 'Watt2T1A2', 'Watt2T2A2', 'Steph1T1', 'Steph1T2',
            'Steph1T3', 'Steph2T1A1', 'Steph2T2A1', 'Steph3T1A1', 'Steph3T2A1', 'Steph3T1A2', 
            'Steph3T2A2', 'Steph2T1A2', 'Steph2T2A2']
eight_bar = list(set(KVdict.keys()) - set(four_bar) - set(six_bar))

for mechType in six_bar:
    batchImg = []
    result_zSet = []
    result_featSet = []
    value = KVdict[mechType]
    z_folder = './outputs-z/'
    e_folder = './outputs-encoded/'
    imgStrings = glob(image_folder + mechType + '/*')
    print(mechType, len(imgStrings))
    for i in tqdm(range(min(setSize, len(imgStrings)))): 
        batchImg.append(cv2.imread(imgStrings[i], cv2.IMREAD_GRAYSCALE)/ 255) # This /255 works better than not doing it 
        floats_before, letter_string, floats_after = process_string_mech(imgStrings[i], toNpy = False)
        if len(floats_after) == 6:
            floats_after = floats_after + [0, 0, 1]
        elif len(floats_after) != 9: 
            print('what?', floats_after)
        
        result_featSet.append(np.array(floats_before + [KVdict[letter_string]] + floats_after, dtype= float).flatten().tolist())
        if len(batchImg) >= batchSize:
            images = torch.from_numpy(np.array([batchImg])).swapaxes(0,1).float().to(device)
            x = model.encoder(images)
            mean, logvar = x[:, : model.latent_dim], x[:, model.latent_dim :]
            z = model.reparameterize(mean, logvar)
            z = z.cpu().detach().numpy()
            result_zSet.append(z)
            batchImg = []

    if len(batchImg) > 0:
        images = torch.from_numpy(np.array([batchImg])).swapaxes(0,1).float().to(device)
        x = model.encoder(images)
        mean, logvar = x[:, : model.latent_dim], x[:, model.latent_dim :]
        z = model.reparameterize(mean, logvar)
        z = z.cpu().detach().numpy()
        result_zSet.append(z)
        batchImg = []

    if len(result_zSet) > 0:
        result_zSet = np.concatenate(result_zSet)
        date = '062324'

        batchZname = z_folder + date + '-z-' + str(int(KVdict[mechType]))
        batchEname = e_folder + date + '-encoded-' + str(int(KVdict[mechType]))
        np.save(batchZname, np.array(result_zSet))
        np.save(batchEname, np.array(result_featSet))

Watt1T1A1 21


  0%|          | 0/21 [00:00<?, ?it/s]

100%|██████████| 21/21 [00:00<00:00, 2629.27it/s]


what? [-8.97, -6.076, -2.308, -2.684, -3.672, -6.697, -7.167, -1.136, -9.162, 7.488, 3.395, -2.455, 9.035, -6.345, 4.711, -0.238, 0.061, -1.342, -0.061, -0.238, -0.471, 0.0, 0.0, 1.0]
what? [5.025, -7.393, -1.63, -5.362, -9.128, 2.599, 4.324, -1.805, 8.738, -7.403, 8.117, 8.784, 7.383, -4.241, -2.817, 0.295, -0.219, 2.727, -0.219, -0.295, -0.158, 0.0, 0.0, 1.0]
what? [1.73, 0.981, 0.305, -0.63, -4.057, 8.77, -9.663, -5.501, 1.657, 7.623, 2.576, -8.282, 3.827, 7.777, -0.213, 0.157, 0.153, 0.93, -0.153, 0.157, 1.571, 0.0, 0.0, 1.0]
what? [4.882, 2.263, -1.785, 4.719, -3.952, -6.348, -4.054, -3.008, 2.802, 6.859, -5.769, -9.77, 3.305, -8.559, -8.594, -0.339, -0.034, -0.407, -0.034, 0.339, 1.051, 0.0, 0.0, 1.0]
what? [-3.145, -0.976, -8.783, -3.962, -7.213, 1.651, 4.983, 6.25, 1.705, -4.848, 7.88, -6.312, -1.391, 0.108, 7.81, -0.183, -0.081, -0.811, 0.081, -0.183, 1.537, 0.0, 0.0, 1.0]
what? [-4.71, -6.75, -6.12, 5.56, 9.145, 1.014, -5.67, -4.436, -6.909, -2.228, 0.68, 9.77, 9.76, -4.824, 

0it [00:00, ?it/s]


...

100%|██████████| 7/7 [00:00<00:00, 2648.16it/s]


what? [5.664, 4.202, 1.993, -9.898, -8.593, -5.596, 6.222, -7.004, -8.254, -7.928, 8.614, 8.297, 0.748, 3.225, -4.895, -0.263, -0.351, 2.257, -0.351, 0.263, 3.656, 0.0, 0.0, 1.0]
what? [7.521, -6.734, 1.6, 2.543, -5.065, -7.762, 3.656, -8.179, -6.325, 1.663, -6.562, 0.228, -7.117, -1.362, -5.246, 0.361, 0.007, 0.722, 0.007, -0.361, -1.104, 0.0, 0.0, 1.0]
what? [1.428, 7.41, -0.378, -1.18, -3.045, 9.99, -9.22, 6.573, 6.306, 4.828, 8.384, 0.662, 0.864, -1.569, 6.324, 0.927, 0.585, -0.527, -0.585, 0.927, -4.284, 0.0, 0.0, 1.0]
what? [-4.762, -5.742, -1.571, -1.282, 4.906, 7.42, -4.003, 4.721, 4.46, 5.841, -5.647, -4.369, -2.641, 8.341, 3.577, -0.788, -0.341, 7.319, 0.341, -0.788, 0.613, 0.0, 0.0, 1.0]
what? [1.935, 3.003, -1.157, 5.249, -8.246, -4.607, -8.14, -4.139, 6.54, -9.739, -2.092, -3.291, 3.863, -3.322, -1.981, -0.174, -0.797, -0.715, 0.797, -0.174, 4.074, 0.0, 0.0, 1.0]
what? [-2.803, -3.095, 3.607, -5.461, 8.921, 1.909, 1.57, 0.182, 7.565, -6.675, -5.27, -2.74, -4.587, -3.456, 8

0it [00:00, ?it/s]


Steph2T2A2 0


0it [00:00, ?it/s]


In [12]:
length = len(result_featSet[0])
for thing in result_featSet:
    if len(thing) != length:
        print(thing, len(thing))



IndexError: list index out of range